### Importing the Libraries


In [377]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importing the dataset


In [378]:
dataset = pd.read_csv('dataset/dataset.csv', header = None)
dataset.columns = dataset.iloc[0]
dataset = dataset[1:]
dataset = dataset.drop(columns=['Year'])

### Encoding Categorical Data


In [379]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
dataset['District'] = label_encoder.fit_transform(dataset['District'])

### Handling Missing Values


In [380]:
dataset = dataset.apply(pd.to_numeric, errors='coerce')
# dataset.interpolate(method='linear', inplace=True)
# dataset.fillna(method='ffill', inplace=True)
# dataset.fillna(dataset.mode(), inplace=True)
# dataset.ffill(inplace=True)
# dataset.fillna(dataset.median(), inplace=True)
# dataset.fillna(dataset.mode(), inplace=True)
dataset.fillna(dataset.mean(), inplace=True)

dataset['Total Cases'] = dataset.iloc[:, 37:50].sum(axis=1)
dataset = dataset.drop(dataset.columns[37:49], axis=1)

In [381]:
print(dataset)

0    District  Temperature_1  Temperature_2  Temperature_3  Temperature_4  \
1           0      20.000000      22.680000      26.655000      29.470000   
2           0      19.385000      21.440000      26.865000      29.515000   
3           0      20.640000      23.015000      28.825000      30.685000   
4           0      18.301344      20.119196      27.013710      29.800000   
5           0      18.979301      22.850893      25.262500      28.902778   
..        ...            ...            ...            ...            ...   
380        63      17.180000      19.535000      23.945000      25.630000   
381        63      17.235000      20.965000      24.980000      27.810000   
382        63      15.633737      16.521280      24.450269      25.818472   
383        63      15.571102      20.074256      23.375269      26.259583   
384        63      20.700000      19.900000      23.800000      31.900000   

0    Temperature_5  Temperature_6  Temperature_7  Temperature_8  \
1       

### Handling Missing Values : Interpolation

In [382]:
# dataset = dataset.apply(pd.to_numeric, errors='coerce')
# dataset.interpolate(method='linear', inplace=True)
# dataset['Total Cases'] = dataset.iloc[:, 37:50].sum(axis=1)
# dataset = dataset.drop(dataset.columns[37:49], axis=1)

### Test/Train Split


In [383]:
from sklearn.model_selection import train_test_split

X = dataset.iloc[:, 0:-1].values
Y = dataset.iloc[:, -1].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)

### Hyperparameter Tuning


#### Bayesian Optimization


In [384]:
# import optuna
# import xgboost as xgb
# from sklearn.metrics import mean_squared_error

# # Define the objective function for Optuna
# def objective(trial):
#     # Suggest hyperparameters
#     param = {
#         "n_estimators": trial.suggest_int("n_estimators", 50, 500),
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
#         "max_depth": trial.suggest_int("max_depth", 3, 10),
#         "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
#         "subsample": trial.suggest_float("subsample", 0.5, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
#         "gamma": trial.suggest_float("gamma", 0.0, 5.0),
#         "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
#         "reg_lambda": trial.suggest_float("reg_lambda", 1.0, 10.0),
#     }

#     # Create and train the model with suggested parameters
#     model = xgb.XGBRegressor(**param, random_state=42, n_jobs=-1)
#     model.fit(X_train, Y_train)

#     # Make predictions and calculate RMSE
#     y_pred = model.predict(X_test)
#     rmse = mean_squared_error(Y_test, y_pred, squared=False)

#     return rmse

# # Create an Optuna study and optimize it
# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=50)

# # Print the best parameters and RMSE
# print(f"Best parameters: {study.best_params}")
# print(f"Best RMSE: {study.best_value}")



#### Random Search


In [385]:
# import xgboost as xgb
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.metrics import mean_squared_error

# # Define the parameter grid for Random Search
# param_grid = {
#     'n_estimators': [50, 100, 200, 300, 500],
#     'learning_rate': np.linspace(0.01, 0.3, 10),
#     'max_depth': [3, 5, 7, 9, 10],
#     'min_child_weight': [1, 3, 5, 7, 10],
#     'subsample': np.linspace(0.5, 1.0, 6),
#     'colsample_bytree': np.linspace(0.5, 1.0, 6),
#     'gamma': np.linspace(0, 5, 10),
#     'reg_alpha': np.linspace(0, 1, 5),
#     'reg_lambda': np.linspace(1, 10, 5),
# }

# # Initialize the XGBoost regressor
# xgb_model = xgb.XGBRegressor(random_state=42, n_jobs=-1)

# # Initialize RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     estimator=xgb_model,
#     param_distributions=param_grid,
#     n_iter=50,  # Number of random combinations to try
#     scoring='neg_mean_squared_error',  # Metric for evaluation
#     cv=5,  # 5-fold cross-validation
#     verbose=1,
#     random_state=42,
#     n_jobs=-1,  # Utilize all CPU cores
# )

# # Fit RandomizedSearchCV
# random_search.fit(X_train, Y_train)

# # Get the best parameters and model performance
# print(f"Best parameters: {random_search.best_params_}")
# print(f"Best RMSE: {np.sqrt(-random_search.best_score_)}")

# # Evaluate the model on the test set
# best_model = random_search.best_estimator_
# y_pred = best_model.predict(X_test)
# test_rmse = mean_squared_error(Y_test, y_pred, squared=False)
# print(f"Test RMSE: {test_rmse}")

### Training Model


In [386]:
import xgboost as xgb
# regressor = xgb.XGBRegressor(colsample_bytree=1, learning_rate=0.042222222222222223,
#                            max_depth=7, alpha=10, n_estimators=500, min_child_weight=1, subsample=0.5, gamma=3.3333333333333335, reg_alpha=0, reg_lambda=1)

# regressor = xgb.XGBRegressor(colsample_bytree=0.7857872949109472, learning_rate=0.010344501343048346,
#                            max_depth=10, alpha=10, n_estimators=257, min_child_weight=2, subsample=0.850598292345319, gamma=3.1518448315907417, reg_alpha=0.9279458085493363, reg_lambda=7.180908650941852)

# regressor = xgb.XGBRegressor(colsample_bytree=0.7, learning_rate=0.042222222222222223,
#                            max_depth=10, alpha=10, n_estimators=50, min_child_weight=1, subsample=1, gamma=2.7777777777777777, reg_alpha=0.5, reg_lambda=10)

regressor = xgb.XGBRegressor()

regressor.fit(X, Y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

#### Comparing Values


In [387]:
Y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2, suppress=True)
print(np.concatenate((Y_pred.reshape(len(Y_pred),1), Y_test.reshape(len(Y_test),1)),1))


[[  338.2    336.  ]
 [  499.7    493.  ]
 [ 6933.48  6950.  ]
 [ 1448.6   1448.  ]
 [ 2844.24  2841.  ]
 [  496.74   491.  ]
 [   25.78    27.  ]
 [    4.38     1.  ]
 [   15.17    10.  ]
 [  516.69   507.8 ]
 [ 3634.67  3627.8 ]
 [  347.69   360.18]
 [  365.65   365.18]
 [   95.56    96.  ]
 [  141.29   140.  ]
 [    5.22    10.  ]
 [ 5848.43  5850.8 ]
 [  758.21   762.  ]
 [ 2360.05  2357.8 ]
 [   64.62    67.  ]
 [    4.75     0.  ]
 [  317.01   318.  ]
 [   -1.8      0.  ]
 [  395.63   391.18]
 [  373.53   374.18]
 [ 4529.71  4533.  ]
 [  357.85   361.  ]
 [  572.37   571.18]
 [    0.19     0.  ]
 [   -6.22     0.  ]
 [ 6577.71  6578.8 ]
 [  922.9    913.18]
 [    8.59     0.  ]
 [  694.6    696.8 ]
 [   -2.87     0.  ]
 [  523.52   525.18]
 [  336.81   335.  ]
 [  -10.58     0.  ]
 [ 1057.56  1053.  ]
 [ 1391.83  1392.8 ]
 [   82.87    84.  ]
 [   14.88    17.  ]
 [  734.39   739.  ]
 [   66.42    62.  ]
 [  491.42   494.18]
 [   -2.63     4.  ]
 [ 3162.73  3195.18]
 [  257.18   

In [388]:
from sklearn.metrics import r2_score
r2_score(Y_test, Y_pred)

0.9999941202271666

In [389]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true, dtype=float), np.array(y_pred, dtype=float)
    non_zero_indices = y_true != 0
    return np.mean(np.abs((y_true[non_zero_indices] - y_pred[non_zero_indices]) / y_true[non_zero_indices])) * 100


mean_absolute_percentage_error(Y_test, Y_pred)

10.154242486079152